<h1 style="color:rgb(0,120,170)">Numpy RNN</h1>


In the notebook, I will implement the recurrent neural network (RNN) using only NumPy functionality. Concretely, I will implement implement the forward and backward pass through the RNN and validate the implementation doing numerical gradient checking.

In [3]:
from typing import Dict, Tuple

import numpy as np

print('python version: 3.9.12')
print('numpy version:',np.__version__)

python version: 3.9.12
numpy version: 1.21.5


<h2 style="color:rgb(0,120,170)">Task 1: Implement Recurrent Neural Network </h2>

In this task you I will implement a recurrent neural network (RNN) in a sequence-to-one setting.

Forward equations of the RNN:
$$
        \boldsymbol{s}(t) = \boldsymbol{W}^T \boldsymbol{x}(t) + \boldsymbol{R}^T \boldsymbol{a}(t-1) + \boldsymbol{b}_s\\
        \boldsymbol{a}(t) = \operatorname{tanh}(\boldsymbol{s}(t)) \\
        \hat{\boldsymbol{y}}(t) = \boldsymbol{V}^T \boldsymbol{a}(t) + \boldsymbol{b}_y
$$

In [4]:
class RNN(object):
    """Numpy implementation of sequence-to-one recurrent neural network for regression tasks."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):

        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # create and initialize weights of the network
        
        self.W = np.random.uniform(-0.2,0.2,size = (self.input_size,self.hidden_size))
        self.R = np.random.uniform(-0.2,0.2,size = (self.hidden_size,self.hidden_size))
        self.bs = np.zeros((self.hidden_size,1))
        self.V = np.random.uniform(-0.2,0.2,size = (self.hidden_size,self.output_size))
        self.by = np.zeros((self.output_size,1))
        self.reset_parameters()

        # place holder to store intermediates for backprop
        self.a = {0:np.zeros((self.hidden_size,1))}
        self.y_hat = None
        self.grads = None
        self.x = None

    def forward(self, x: np.ndarray) -> np.ndarray:

        self.x = np.expand_dims(x,axis = 2)
        for i in range(len(x)):
            s_t = self.W.T @ self.x[i] + self.R.T @ self.a[i] + self.bs
            a_t = np.tanh(s_t)
            self.a[i + 1] = a_t
            
        
        self.y_hat = self.V.T @ a_t + self.by
        
            
        return self.y_hat

    def backward(self, d_loss: np.ndarray) -> Dict:
      
        delta_t = np.zeros((1,self.hidden_size))
        
        dV = np.zeros(self.V.T.shape)
        dby = np.zeros((self.output_size,1)).T
        
        dW = np.zeros(self.W.T.shape)
        dR = np.zeros(self.R.T.shape)
        dbs = np.zeros(self.bs.T.shape)
        
        dV = d_loss @ self.a[len(self.a)-1].T
        dby += d_loss.T

        for i in reversed(range(1,len(self.a))):
            
            if i == len(self.a) - 1:
                delta_t = (d_loss.T @ self.V.T + delta_t @ self.R.T) @ np.diag(np.diag(1 - self.a[i] @ self.a[i].T))
            else:
                delta_t = (delta_t @ self.R.T) @ np.diag(np.diag(1 - self.a[i] @ self.a[i].T))
        
            dW += delta_t.T @ self.x[i-1].T
            dR += delta_t.T @ self.a[i-1].T
            dbs += delta_t
            
        self.grads = {'dW': dW, 'dR': dR, 'dV': dV, 'dbs': dbs, 'dby': dby}
        return self.grads
        
        
    def update(self, lr: float):

        if not self.grads:
            raise RuntimeError("You have to call the .backward() function first")
            

        self.V -= lr * self.grads['dV'].T
        self.W -= lr * self.grads['dW'].T
        self.R -= lr * self.grads['dR'].T
        self.bs -= lr * self.grads['dbs'].T
        self.by -= lr * self.grads['dby'].T
        # reset internal class attributes
        self.grads = {}
        self.y_hat, self.a = None, {0:np.zeros((self.hidden_size,1))}
        
    def get_weights(self) -> Dict:
        return {'W': self.W, 'R': self.R, 'V': self.V, 'bs': self.bs, 'by': self.by}
    
    def set_weights(self, weights: Dict):
        if not all(name in weights.keys() for name in ['W', 'R', 'V']):
            raise ValueError("Missing one of 'W', 'R', 'V' keys in the weight dictionary")
        self.W = weights["W"]
        self.R = weights["R"]
        self.V = weights["V"]
        self.bs = weights["bs"]
        self.by = weights["by"]
        
    def reset_parameters(self):
        # recurrent weights initialized as identity matrix
        self.R = np.eye(self.hidden_size)
        
        # input to hidden and hidden to output initialized with LeCun initialization
        gain = np.sqrt(3 / self.input_size)
        self.W = np.random.uniform(-gain, gain, self.W.shape)
        gain = np.sqrt(3 / self.hidden_size)
        self.V = np.random.uniform(-gain, gain, self.V.shape)

<h2 style="color:rgb(0,120,170)">Task 2: Numerical gradient check</h2>

To validate your implementation of the backward pass use the two-sided gradient approximation

$$
\frac{\partial f(w_1, \dots, w_n)}{\partial w_i} \approx \frac{f(w_1, \dots, w_i + \epsilon, \dots, w_n) - f(w_1, \dots, w_i - \epsilon, \dots, w_n)}{2*\epsilon}.
$$

In [5]:
def get_numerical_gradient(model: RNN, x: np.ndarray, eps: float=1e-7) -> Dict:
    """Implementation of the two-sided numerical gradient approximation
    
    Parameters
    ----------
    model : RNN
        The RNN model object
    x : np.ndarray
        Input sequence(s) of shape [sequence length, number of features]
    eps : float
        The epsilon used for numerical gradient approximation
        
    Returns
    -------
    A dictionary containing the numerical gradients for each weight of the RNN. Make sure
    to name the dictionary keys like the names of the RNN gradients dictionary (e.g. 
    'd_R' for the weight 'R')
    """  
    weights = model.get_weights()
    num_grad = {}
    
    for name_layer,weights_layer in weights.items():#iterate through each layer
        gradient = np.zeros((weights_layer.shape))
        for i in range(weights_layer.shape[0]):
            for j in range(weights_layer.shape[1]):
                weights_plus = weights.copy()
                weights_min = weights.copy()
                
                #changing one element in weight layer:
                w_min = np.copy(weights_layer)
                w_min[i,j] = w_min[i,j] - eps

                w_plus = np.copy(weights_layer)
                w_plus[i,j] = w_plus[i,j] + eps; 
                
                #computing forward passes for plus and minus epsilons
                weights_min[name_layer] = w_min
                model.set_weights(weights_min)
                forward_min = model.forward(x)
                
                weights_plus[name_layer] = w_plus
                model.set_weights(weights_plus)
                forward_plus = model.forward(x)
                
                gradient[i,j] = np.sum((forward_plus - forward_min)/(2*eps))
         
                
        
        num_grad['d' + name_layer] = gradient.T        
    return num_grad


def get_analytical_gradient(model: RNN, x: np.ndarray) -> Dict:
    """Helper function to get the analytical gradient.
        
    Parameters
    ----------
    model : RNN
        The RNN model object
    x : np.ndarray
        Input sequence(s) of shape [sequence length, number of features]
        
    Returns
    -------
    A dictionary containing the analytical gradients for each weight of the RNN.
    """     
    e_t = np.ones((model.output_size,1))
        
    res = model.forward(x)
    grads = model.backward(e_t)
    return grads

            
def gradient_check(model: RNN, x: np.ndarray, treshold: float = 1e-7):
    """Perform gradient checking.
    
    You don't have to do anything in this function.
    
    Parameters
    ----------
    model : RNN
        The RNN model object
    x : np.ndarray
        Input sequence(s) of shape [sequence length, number of features]
    eps : float
        The epsilon used for numerical gradient approximation    
    """
    numerical_grads = get_numerical_gradient(model, x)
    analytical_grads = get_analytical_gradient(model, x)
    
    for key, num_grad in numerical_grads.items():
        difference = np.linalg.norm(num_grad - analytical_grads[key])
        if difference < treshold:
            print(f"Gradient check for {key} passed (difference {difference:.3e})")
        else:
            print(f"Gradient check for {key} failed (difference {difference:.3e})")

Now we use the `gradient_check` function to test your RNN implementation for two different scenarios, which are a single output neuron and multiple output neurons. 

In [6]:
print("Gradient check with a single output neuron:")
output_size = 1
model = RNN(input_size=5, hidden_size=10, output_size=output_size)
x = np.random.rand(5, 5)
gradient_check(model, x)

print("\nGradient check with multiple output neurons:")
output_size = 5
model = RNN(input_size=5, hidden_size=10, output_size=output_size)
x = np.random.rand(5, 5)
gradient_check(model, x)

Gradient check with a single output neuron:
Gradient check for dW passed (difference 2.035e-09)
Gradient check for dR passed (difference 2.565e-09)
Gradient check for dV passed (difference 1.133e-09)
Gradient check for dbs passed (difference 8.493e-10)
Gradient check for dby passed (difference 2.876e-11)

Gradient check with multiple output neurons:
Gradient check for dW passed (difference 4.929e-09)
Gradient check for dR passed (difference 6.825e-09)
Gradient check for dV passed (difference 2.638e-09)
Gradient check for dbs passed (difference 1.614e-09)
Gradient check for dby passed (difference 7.877e-10)
